In [1]:
import pandas as pd
from collections import Counter

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
inputfile = "/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/cnn_data_corpus.csv"

In [3]:
torch.manual_seed(1)

In [15]:
qdf = pd.read_csv(inputfile)

/home/francisco/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
query = qdf['query_id']
true_summaries = qdf['tstokens']
sentences = qdf[[x for x in qdf.columns if 'stokens_' in x]]

In [35]:
corpus = Counter()
for word in ['a' ,'b', 'c', 'a', 'd']:
    corpus[word] += 1

In [37]:
corpus

Counter({'a': 2, 'b': 1, 'c': 1, 'd': 1})

In [40]:
x = [1, 2, 3]
y = [4, 5, 6, 1]

In [78]:
def rougueScores(genSummary, refSummary):
    genTotal, refTotal, intersection = 0, 0, 0
    for token in list(set(list(refSummary.keys()) + list(genSummary.keys()) )):
        intersection += min(refSummary[token], genSummary[token])
        refTotal += refSummary[token]
        genTotal += genSummary[token]

    recall = intersection / float(refTotal) if float(refTotal) > 0 else 0
    prec   = intersection / float(genTotal) if float(genTotal) > 0 else 0
    f1 = (2 * recall * prec) / (recall + prec) if (recall + prec) > 0 else 0
    
    return recall, prec, f1

In [79]:
trueSummary = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3])

predSummary0 = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3])

predSummary1 = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3, 4])

predSummary2 = Counter([1, 1, 1, 2, 2, 2, 2, 3])

predSummary3 = Counter([4])


In [80]:
print(rougueScores(predSummary0, trueSummary))
print(rougueScores(predSummary1, trueSummary))
print(rougueScores(predSummary2, trueSummary))
print(rougueScores(predSummary3, trueSummary))

(1.0, 1.0, 1.0)
(1.0, 0.9, 0.9473684210526316)
(0.8888888888888888, 1.0, 0.9411764705882353)
(0.0, 0.0, 0)


In [4]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [5]:
class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is num_labels!
        self.linear = nn.Linear(vocab_size, num_labels)

        # NOTE! The non-linearity log softmax does not have parameters! So we don't need
        # to worry about that here

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec))


def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [6]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [7]:
# the model knows its parameters.  The first output below is A, the second is b.
# Whenever you assign a component to a class variable in the __init__ function
# of a module, which was done with the line
# self.linear = nn.Linear(...)
# Then through some Python magic from the Pytorch devs, your module
# (in this case, BoWClassifier) will store knowledge of the nn.Linear's parameters
for param in model.parameters():
    print(param)

# To run the model, pass in a BoW vector, but wrapped in an autograd.Variable
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

Parameter containing:

Columns 0 to 9 
-0.0325  0.1950  0.0864  0.1697 -0.1961 -0.1459 -0.0775  0.1957 -0.1386 -0.1035
 0.1483 -0.1061 -0.1854  0.0135  0.0669  0.1624 -0.0324 -0.0168  0.0230 -0.0272

Columns 10 to 19 
-0.1599 -0.0406 -0.1231 -0.0440 -0.0606  0.0666 -0.0405  0.1708  0.0152  0.1358
-0.1411  0.1722 -0.1184  0.1092  0.1180  0.0847  0.1837  0.1188 -0.0732 -0.1597

Columns 20 to 25 
-0.0317 -0.0732  0.0726  0.0096 -0.1159 -0.0222
 0.0754  0.0071  0.1476  0.1432  0.1548  0.1291
[torch.FloatTensor of size 2x26]

Parameter containing:
-0.1628
 0.1293
[torch.FloatTensor of size 2]

Variable containing:
-0.8630 -0.5480
[torch.FloatTensor of size 1x2]



In [8]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [9]:
# Run on test data before we train, just to see a before-and-after
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-1.2611 -0.3332
[torch.FloatTensor of size 1x2]

Variable containing:
-1.1140 -0.3978
[torch.FloatTensor of size 1x2]



In [10]:
# Print the matrix column corresponding to "creo"
print(next(model.parameters())[:, word_to_ix["creo"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

Variable containing:
-0.1599
-0.1411
[torch.FloatTensor of size 2]



In [11]:
# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Variable as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-0.2209 -1.6186
[torch.FloatTensor of size 1x2]

Variable containing:
-3.0979 -0.0462
[torch.FloatTensor of size 1x2]



In [12]:
# Index corresponding to Spanish goes up, English goes down!
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
 0.3315
-0.6325
[torch.FloatTensor of size 2]

